In [1]:
#Import packages
from selenium import webdriver

from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.expected_conditions import visibility_of_element_located
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By

from selenium.webdriver.support.wait import WebDriverWait

from datetime import datetime
import pandas as pd
import numpy as np
import random
import time
import os
import re

In [2]:
os.makedirs('recipe_data/', exist_ok=True)

In [ ]:
options = webdriver.ChromeOptions()
capabilities = options.to_capabilities()
capabilities = {
 'browser': 'chrome',
 'browser_version': 'latest',
 'os': 'Windows',
 'os_version': '10',
 'build': 'Python Sample Build',
 'name': 'Pop-ups testing'
}
#capabilities["chromeOptions"]["excludeSwitches"] = ["disable-popup-blocking"]

driver = webdriver.Chrome(ChromeDriverManager().install(), options=options)
driver.maximize_window()

url = "https://panlasangpinoy.com/categories/recipes/page/64"

driver.get(url)
driver.implicitly_wait(8)

page = 64

while True:
    try:
        print(f'Harvesting page: {page}....', end="")
        
        urls = []
        
        total_height = int(driver.execute_script("return document.body.scrollHeight"))
        for i in range(1, total_height, 25):
            driver.execute_script("window.scrollTo(0, {});".format(i))

        recipe = driver.find_elements(By.XPATH, '//div[@class = "entry-content"]//a[@class="entry-image-link"]')
        for _recipe in recipe:
            urls.append({"url":_recipe.get_attribute('href')})
            time.sleep(0.5)
            
        url_df = pd.DataFrame(urls)
        url_df.to_csv("recipe_data/panlasang_pinoy_" + str(page) + ".csv", index=False)
        print("Saved!")
        
        start_url = driver.current_url
        time.sleep(0.5)

        next_page = driver.find_element(By.XPATH, "//li[@class='pagination-next']")
        next_page.click()
        time.sleep(1)
        
        
        time.sleep(1)
        
        next_url = driver.current_url
        if start_url == next_url:
            print(f"Scraper stopped moving, scaping ended at page {page}")
            break

        page += 1
        
    except:
        print("Scraping stopped moving")
        break



In [3]:
url = pd.DataFrame({"url": []})
                    
for i in range(1,79):
    df = pd.read_csv("recipe_data/panlasang_pinoy_"+str(i)+".csv")
    url = pd.concat([url, df], axis=0)
    data_url = url.reset_index(drop=True)

In [7]:
data_url[data_url['url'] == 'https://panlasangpinoy.com/asian-food-dish-steamed-fish-lapu-lapu-recipe/']

,url
1758,https://panlasangpinoy.com/asian-food-dish-ste...


In [6]:
driver = webdriver.Chrome(ChromeDriverManager().install())
recipes = []
item = 0

for food in data_url["url"]:
    driver.get(food)
    time.sleep(1)
    
    try:
        meal = driver.find_element(By.XPATH,"//div[@class='wprm-recipe wprm-recipe-template-classic']//h2[@class='wprm-recipe-name wprm-block-text-bold']").text
    except:
        meal = "none"
    try:
        course = driver.find_element(By.XPATH,"//div[@class='wprm-recipe wprm-recipe-template-classic']//span[@class='wprm-recipe-course wprm-block-text-normal']").text
    except:
        course = 'none'
    #keyword = driver.find_element(By.XPATH,"//div[@class='wprm-recipe wprm-recipe-template-classic']//span[@class='wprm-recipe-keyword wprm-block-text-normal']").text
    #prep_and_cook_time = driver.find_elements(By.XPATH,"//div[@class='wprm-recipe wprm-recipe-template-classic']//span[@class='wprm-recipe-time wprm-block-text-normal']")
    #cook_time = driver.find_element(By.XPATH,"//div[@class='wprm-recipe wprm-recipe-template-classic']//span[@class='wprm-recipe-details wprm-recipe-details-minutes wprm-recipe-cook_time wprm-recipe-cook_time-minutes']").text
    #number_of_serving = driver.find_element(By.XPATH,"//div[@class='wprm-recipe wprm-recipe-template-classic']//a[@class='wprm-recipe-servings-link']").text
    #calories = driver.find_element(By.XPATH,"//div[@class='wprm-recipe wprm-recipe-template-classic']//span[@class='wprm-recipe-details wprm-recipe-nutrition wprm-recipe-calories wprm-block-text-normal']").text
    try:
        ingredients_list = driver.find_elements(By.XPATH,"//div[@class='wprm-recipe wprm-recipe-template-classic']//li[@class='wprm-recipe-ingredient']")
        ingredients = []
        for i in range(0, len(ingredients_list)):
            ingredients.append(ingredients_list[i].text)
        ingredients = ', '.join(ingredients)
        
    except:
        ingredients = "none"
        
    try:
        instructions = driver.find_element(By.XPATH,"//div[@class='wprm-recipe wprm-recipe-template-classic']//ul[@class='wprm-recipe-instructions']").text
    except:
        instructions = 'none'
        
    
    recipes.append({
        "url": food,
        "food": meal,
        "course": course,
        #"keyword": keyword, 
        #"prep time": prep_and_cook_time[0].text,
        #"cook time": prep_and_cook_time[1].text,
        #"number of serving": number_of_serving, 
        #"calories": calories,
        "ingredients": ingredients,
        "instructions": instructions
        })
    time.sleep(1.5)
    item += 1
    
    if item % 400 == 0:
        driver.quit()
        driver = webdriver.Chrome(ChromeDriverManager().install())
        pass

final_data = pd.DataFrame(recipes)
final_data.to_csv("recipe_data/panlasang_pinoy.csv", index=False)



====== WebDriver manager ======
Current google-chrome version is 109.0.5414
Get LATEST driver version for 109.0.5414
Driver [C:\Users\Emong\.wdm\drivers\chromedriver\win32\109.0.5414.74\chromedriver.exe] found in cache


====== WebDriver manager ======
Current google-chrome version is 109.0.5414
Get LATEST driver version for 109.0.5414
Get LATEST driver version for 109.0.5414
Trying to download new driver from https://chromedriver.storage.googleapis.com/109.0.5414.74/chromedriver_win32.zip
Driver has been saved in cache [C:\Users\Emong\.wdm\drivers\chromedriver\win32\109.0.5414.74]


====== WebDriver manager ======
Current google-chrome version is 109.0.5414
Get LATEST driver version for 109.0.5414
Driver [C:\Users\Emong\.wdm\drivers\chromedriver\win32\109.0.5414.74\chromedriver.exe] found in cache


====== WebDriver manager ======
Current google-chrome version is 109.0.5414
Get LATEST driver version for 109.0.5414
Driver [C:\Users\Emong\.wdm\drivers\chromedriver\win32\109.0.5414.74\

TimeoutException: Message: timeout: Timed out receiving message from renderer: -4440.882
  (Session info: chrome=109.0.5414.75)


In [10]:
final_data = pd.DataFrame(recipes)
final_data.to_csv("recipe_data/panlasang_pinoy.csv", index=False)